In [1]:
import sys
import torch as t
sys.path.append("../../")
from periodic_padding import periodic_padding

In [100]:
def split_grids(batch:t.Tensor, kernel_width: int, kernel_height: int) -> t.Tensor:
    unfold_transform = t.nn.Unfold(kernel_size=(kernel_height,kernel_width))
    reflective_padding = t.nn.ReflectionPad2d(1)
    
    padded_batch = reflective_padding(batch).float()
    padded_batch = padded_batch.unsqueeze(1)
    
    return t.transpose(unfold_transform(padded_batch), dim0=1, dim1=2)

## sanity check split

tensor([[[ 5.,  4.,  5.,  6.,  7.,  6.],
         [ 1.,  0.,  1.,  2.,  3.,  2.],
         [ 5.,  4.,  5.,  6.,  7.,  6.],
         [ 9.,  8.,  9., 10., 11., 10.],
         [13., 12., 13., 14., 15., 14.],
         [ 9.,  8.,  9., 10., 11., 10.]]])


In [55]:
def tile_idx2conv_idx(row, col, tile_size, batch_size):
    _, batch_height, batch_width = batch_size
    tile_height, tile_width = tile_size
    tiles_per_row = batch_width-tile_width+1
    return int(tile_height*row*tiles_per_row + tile_width*col)

## sanity checks for the helper functions

In [61]:
test = t.arange(64).reshape(1,8,8).float()
print(test)

split = split_grids(test, kernel_width=4, kernel_height=4)
print(split.shape)
assert split.shape == (1, 25, 16)

row = 1
col = 2
idx = tile_idx2conv_idx(row,col,(4,4),test.shape)
print(idx)
assert idx == 18
tile = split[0,idx]
assert t.all(tile == t.tensor([
    11, 12, 13, 14,
    19, 20, 21, 22,
    27, 28, 29, 30,
    35, 36, 37, 38
]))

row = 2
col = 0
idx = tile_idx2conv_idx(row,col,(4,4),test.shape)
assert idx == 28
tile = split[0,idx]
assert t.all(tile == t.tensor([
    31, 24, 25, 26,
    39, 32, 33, 34,
    47, 40, 41, 42,
    55, 48, 49, 50
]))

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11., 12., 13., 14., 15.],
         [16., 17., 18., 19., 20., 21., 22., 23.],
         [24., 25., 26., 27., 28., 29., 30., 31.],
         [32., 33., 34., 35., 36., 37., 38., 39.],
         [40., 41., 42., 43., 44., 45., 46., 47.],
         [48., 49., 50., 51., 52., 53., 54., 55.],
         [56., 57., 58., 59., 60., 61., 62., 63.]]])
torch.Size([1, 25, 16])
28


AssertionError: 

In [38]:
print(test)

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7],
         [ 8,  9, 10, 11, 12, 13, 14, 15],
         [16, 17, 18, 19, 20, 21, 22, 23],
         [24, 25, 26, 27, 28, 29, 30, 31],
         [32, 33, 34, 35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44, 45, 46, 47],
         [48, 49, 50, 51, 52, 53, 54, 55],
         [56, 57, 58, 59, 60, 61, 62, 63]]])
